# Test Básico para ver si funciona TensorFlow

In [1]:
# Importamos librerias
import tensorflow as tf

In [2]:
# Reseteamos todo para ejecutar en jupyter desde el principio
tf.reset_default_graph()

# Configuracion de hiperparametros
batch_size = 100
learning_rate = 0.5
training_epochs = 100

# defino el directorio de salida de los eventos de Summary
logs_path = "tblogs/"


In [3]:
# Cargamos el MNIST 
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('/home/jorge/data/MNIST', one_hot=True)

Extracting /home/jorge/data/MNIST/train-images-idx3-ubyte.gz
Extracting /home/jorge/data/MNIST/train-labels-idx1-ubyte.gz
Extracting /home/jorge/data/MNIST/t10k-images-idx3-ubyte.gz
Extracting /home/jorge/data/MNIST/t10k-labels-idx1-ubyte.gz


In [4]:
type(mnist)

tensorflow.contrib.learn.python.learn.datasets.base.Datasets

In [5]:
mnist.train.num_examples

55000

In [6]:
mnist.test.num_examples

10000

In [7]:
# Definimos las variables y la entrada de las imagenes

with tf.name_scope('input'):
    # None -> el tamaño del batch puede ser cualquiera, 784 -> aplana la imagen de 28x28
    x = tf.placeholder(tf.float32, shape=[None, 784], name="x-input") 

    # target de 10 clases de salida
    y_ = tf.placeholder(tf.float32, shape=[None, 10], name="y-input")


# Los parametros del modelo deben variar durante el training entonces usamos "tf.Variable"
with tf.name_scope("weights"):
    W = tf.Variable(tf.zeros([784, 10]))

# incluimos el bias
with tf.name_scope("biases"):
    b = tf.Variable(tf.zeros([10]))

# implementamos el modelo
with tf.name_scope("softmax"):
    # y es nuestra prediccion 
    y = tf.nn.softmax(tf.matmul(x,W) + b)

# Especificamos la funcion de coste
with tf.name_scope('cross_entropy'):
    cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

# Especificamos el optimizador
with tf.name_scope('train'):
    # optimizer is an "operation" which we can execute in a session
    train_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)

with tf.name_scope('Accuracy'):
    # Accuracy
    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
# Creamos un summary para la funcion de coste y la accuracy
tf.summary.scalar("cost", cross_entropy)
tf.summary.scalar("accuracy", accuracy)

# Consolida todos los summarys en una unica "operacion" que puede ejecutarse en la sesion.
#summary_op = tf.merge_all_summaries()

summary_op = tf.summary.merge_all()



In [8]:
with tf.Session() as sess:
    # Inicializamos las variables de TF como siempre
    sess.run(tf.initialize_all_variables())

    # creamos el objeto "writer" 
    writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
        
    # Entrenamos
    #----------------------------------------------------------------------------------------
    for epoch in range(training_epochs):
        
        # calculamos el numero de batches en cada epoch
        batch_count = int(mnist.train.num_examples/batch_size)
        
        for i in range(batch_count):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            
            # ejecutar las operaciones definidas en el batch
            _, summary = sess.run([train_op, summary_op], feed_dict={x: batch_x, y_: batch_y})
            
            # escribimos el log
            writer.add_summary(summary, epoch * batch_count + i)
            
        if epoch % 10 == 0: 
            print "Epoch: ", epoch 
            

    print "Accuracy: ", accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels})
    print "Terminado..."

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Epoch:  0
Epoch:  10
Epoch:  20
Epoch:  30
Epoch:  40
Epoch:  50
Epoch:  60
Epoch:  70
Epoch:  80
Epoch:  90
Accuracy:  0.9246
done
